# Linear Algebra

Álgebra linear é o ramo da matemática que lida com espaços vetoriais. 

In [2]:
from __future__ import division # want 3 / 2 == 1.5
import re, math, random # regexes, math functions, random numbers
import matplotlib.pyplot as plt # pyplot
from collections import defaultdict, Counter
from functools import partial

## Vetores

Abstratamente, os vetores são objetos que podem ser somados juntos (para formar novos vetores) e que podem ser multiplicados pelos escalares (por exemplo números), também para formar vetores novos.

Concretamente, vetores são pontos em algum espaço de dimensão finita. Apesar não pensarmos em nossos dados como vetores, eles são uma ótima maneira de representar dados numéricos.

A abordagem mais simples é representar vetores com uma lista de números. O primeiro problema disso é que queremos realizar aritmética nos vetores. Como as listas em python não são vetores, precisoaremos utilizar alguma biblioteca ou construir essas ferramentas. Vamos seguir a segunda opção.

### Functions for working with vectors

Para iniciar, frenquentemente vamos vamos trabalhar com dois ou mais vetores. Os vetores se adicionam item a item. Isso significa que, se dois vetores **v** e **w** possuem o mesmo tamanho, sua soma é somente o vetor cujo primeiro elemento seja **v[0] + w[0]** e assim por diante. Neste caso, se os vetores não possuirem o mesmo tamanho não será possível realizar a soma. Veja a implementação na função **def vector_add(v, w)**.

In [3]:
'''
Podemos implementar este método utilizando vetores zip juntos e usar uma compressão de lista
para adicionar os elementos correspondentes.
'''
def vector_add(v, w):
    """adds two vectors componentwise"""
    return [v_i + w_i for v_i, w_i in zip(v,w)]

Faremos o mesmo processo para subtrair dois vetores, obviamente realizando uma subtração em vez de uma adição.

In [4]:
def vector_subtract(v, w):
    """subtracts two vectors componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]

Para adicionar uma soma entre vetores, ou seja, gerar um vetor resultante, a maneira mais fácil seria adicionar um vetor de cada vez:

```
def vector_sum(vectors):
  result = vectors[0] # ....................começa com o primeiro vetor           
  for vector in vectors[1:]: # .............depois passa por todos os outros
    result = vector_add(result, vector) # ..e os adiciona ao resultado
  return result
```

Se você observar atentamente, o que estamos fazendo é um **reducing** utilizando a operação **vector_add**, o que significa que podemos reescrever de forma reduzida apenas usando funções de alta ordem como em **vector_sum**.

In [5]:
def vector_sum(vectors):
    return reduce(vector_add, vectors)

Outro fator é que queremos poder multiplicar um vetor por um escalar, que simplesmente faremos ao multiplicar cada elemento do vetor pelo número determinado, exatamente como em **scalar_multiply**.

In [6]:
def scalar_multiply(c, v):
    return [c * v_i for v_i in v]

this isn't right if you don't from __future__ import division

In [7]:
def vector_mean(vectors):
    """compute the vector whose i-th element is the mean of the
    i-th elements of the input vectors"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

def magnitude(v):
    return math.sqrt(sum_of_squares(v))

def squared_distance(v, w):
    return sum_of_squares(vector_subtract(v, w))

def distance(v, w):
    return math.sqrt(squared_distance(v, w))

## functions for working with matrices

In [8]:
def shape(A):
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols

def get_row(A, i):
    return A[i]
    
def get_column(A, j):
    return [A_i[j] for A_i in A]

def make_matrix(num_rows, num_cols, entry_fn):
    """returns a num_rows x num_cols matrix 
    whose (i,j)-th entry is entry_fn(i, j)"""
    return [[entry_fn(i, j) for j in range(num_cols)]
            for i in range(num_rows)]  

def is_diagonal(i, j):
    """1's on the 'diagonal', 0's everywhere else"""
    return 1 if i == j else 0

identity_matrix = make_matrix(5, 5, is_diagonal)

In [9]:
#          user 0  1  2  3  4  5  6  7  8  9
#
friendships = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0], # user 0
               [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], # user 1
               [1, 1, 0, 1, 0, 0, 0, 0, 0, 0], # user 2
               [0, 1, 1, 0, 1, 0, 0, 0, 0, 0], # user 3
               [0, 0, 0, 1, 0, 1, 0, 0, 0, 0], # user 4
               [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], # user 5
               [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], # user 6
               [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], # user 7
               [0, 0, 0, 0, 0, 0, 1, 1, 0, 1], # user 8
               [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]] # user 9

## DELETE DOWN

In [10]:
def matrix_add(A, B):
    if shape(A) != shape(B):
        raise ArithmeticError("cannot add matrices with different shapes")
        
    num_rows, num_cols = shape(A)
    def entry_fn(i, j): return A[i][j] + B[i][j]
        
    return make_matrix(num_rows, num_cols, entry_fn)


def make_graph_dot_product_as_vector_projection(plt):

    v = [2, 1]
    w = [math.sqrt(.25), math.sqrt(.75)]
    c = dot(v, w)
    vonw = scalar_multiply(c, w)
    o = [0,0]

    plt.arrow(0, 0, v[0], v[1], 
              width=0.002, head_width=.1, length_includes_head=True)
    plt.annotate("v", v, xytext=[v[0] + 0.1, v[1]])
    plt.arrow(0 ,0, w[0], w[1], 
              width=0.002, head_width=.1, length_includes_head=True)
    plt.annotate("w", w, xytext=[w[0] - 0.1, w[1]])
    plt.arrow(0, 0, vonw[0], vonw[1], length_includes_head=True)
    plt.annotate(u"(v•w)w", vonw, xytext=[vonw[0] - 0.1, vonw[1] + 0.1])
    plt.arrow(v[0], v[1], vonw[0] - v[0], vonw[1] - v[1], 
              linestyle='dotted', length_includes_head=True)
    plt.scatter(*zip(v,w,o),marker='.')
    plt.axis('equal')
    plt.show()

In [11]:
#from linear_algebra import distance, vector_mean
v = [1, 2, 3]
w = [4, 5, 6]

print(distance(v, w))
#print(vector_mean([v, w]))

5.196152422706632
